# Maven Churn Challenge

##### Task Description

You'll be assuming the role of a BI Consultant for Maven Communications, a California-based Telecommunications company.

You've been hired to help the company improve retention by identifying high value customers and churn risks, and have been asked to present your findings to the CMO in the form of a single page report or dashboard.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

###### 1.Read csv files

In [2]:
# Create dataframe to read first csv file
mavencustomer_df = pd.read_csv('/Users/HP 1030/Documents/Portfolio Projects/Maven Challenge/telecom_customer_churn.csv')
mavencustomer_df.head(5)

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [3]:
# Create dataframe to read second csv file
mavendata_df = pd.read_csv('/Users/HP 1030/Documents/Portfolio Projects/Maven Challenge/telecom_data_dictionary.csv', encoding='ISO-8859–1')
mavendata_df.head(5)

,Table,Field,Description
0,Customer Churn,CustomerID,A unique ID that identifies each customer
1,Customer Churn,Gender,"The customers gender: Male, Female"
2,Customer Churn,Age,"The customers current age, in years, at the t..."
3,Customer Churn,Married,"Indicates if the customer is married: Yes, No"
4,Customer Churn,Number of Dependents,Indicates the number of dependents that live w...


In [4]:
# Create dataframe to read third csv file
mavenzip_df = pd.read_csv('/Users/HP 1030/Documents/Portfolio Projects/Maven Challenge/telecom_zipcode_population.csv')
mavenzip_df.head(5)

,Zip Code,Population
0,90001,54492
1,90002,44586
2,90003,58198
3,90004,67852
4,90005,43019


###### 2.Previewing our datasets

In [5]:
# Previewing our first dataset

mavencustomer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Gender                             7043 non-null   object 
 2   Age                                7043 non-null   int64  
 3   Married                            7043 non-null   object 
 4   Number of Dependents               7043 non-null   int64  
 5   City                               7043 non-null   object 
 6   Zip Code                           7043 non-null   int64  
 7   Latitude                           7043 non-null   float64
 8   Longitude                          7043 non-null   float64
 9   Number of Referrals                7043 non-null   int64  
 10  Tenure in Months                   7043 non-null   int64  
 11  Offer                              7043 non-null   objec

In [6]:
# reading second dataset
mavendata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Table        40 non-null     object
 1   Field        40 non-null     object
 2   Description  40 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [7]:
# reading third dataset
mavenzip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Zip Code    1671 non-null   int64
 1   Population  1671 non-null   int64
dtypes: int64(2)
memory usage: 26.2 KB


#### 3.Data Cleaning

###### 3.1Completeness

In [8]:
# check for columns with null in the telecom_customer_churn table
mavencustomer_df.isnull().any()

Customer ID                          False
Gender                               False
Age                                  False
Married                              False
Number of Dependents                 False
City                                 False
Zip Code                             False
Latitude                             False
Longitude                            False
Number of Referrals                  False
Tenure in Months                     False
Offer                                False
Phone Service                        False
Avg Monthly Long Distance Charges     True
Multiple Lines                        True
Internet Service                     False
Internet Type                         True
Avg Monthly GB Download               True
Online Security                       True
Online Backup                         True
Device Protection Plan                True
Premium Tech Support                  True
Streaming TV                          True
Streaming M

###### 3.2 Validity

In [9]:
# Drop columns we wont be using at the moment
new_mavencustomer_df = mavencustomer_df.drop(["Gender","Age","Married","Latitude","Longitude","Payment Method","Paperless Billing"], axis = 1)
new_mavencustomer_df.head(5)

,Customer ID,Number of Dependents,City,Zip Code,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,...,Contract,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,0,Frazier Park,93225,2,9,None,Yes,42.39,No,...,One Year,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,0,Glendale,91206,0,9,None,Yes,10.69,Yes,...,Month-to-Month,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,0,Costa Mesa,92627,0,4,Offer E,Yes,33.65,No,...,Month-to-Month,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,0,Martinez,94553,1,13,Offer D,Yes,27.82,No,...,Month-to-Month,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,0,Camarillo,93010,3,3,None,Yes,7.38,No,...,Month-to-Month,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [10]:
# Remove spaces in column names
def change(new):
    new_mavencustomer_df.columns = new_mavencustomer_df.columns.str.replace(" ","_", regex= True)
    return new_mavencustomer_df.head(5)

change(new_mavencustomer_df)

,Customer_ID,Number_of_Dependents,City,Zip_Code,Number_of_Referrals,Tenure_in_Months,Offer,Phone_Service,Avg_Monthly_Long_Distance_Charges,Multiple_Lines,...,Contract,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,0002-ORFBO,0,Frazier Park,93225,2,9,None,Yes,42.39,No,...,One Year,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,0,Glendale,91206,0,9,None,Yes,10.69,Yes,...,Month-to-Month,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,0,Costa Mesa,92627,0,4,Offer E,Yes,33.65,No,...,Month-to-Month,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,0,Martinez,94553,1,13,Offer D,Yes,27.82,No,...,Month-to-Month,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,0,Camarillo,93010,3,3,None,Yes,7.38,No,...,Month-to-Month,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


###### 3.3 Consistency

In [11]:
# Check for duplicates
duplicate = new_mavencustomer_df[new_mavencustomer_df.duplicated()]
duplicate

,Customer_ID,Number_of_Dependents,City,Zip_Code,Number_of_Referrals,Tenure_in_Months,Offer,Phone_Service,Avg_Monthly_Long_Distance_Charges,Multiple_Lines,...,Contract,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason


In [12]:
# Replace the Nulls in the Churn_Reason column with "No Feedback Given"
new_mavencustomer_df['Churn_Reason'] = new_mavencustomer_df['Churn_Reason'].fillna( 'No Feedback Given')
new_mavencustomer_df.head(5)

,Customer_ID,Number_of_Dependents,City,Zip_Code,Number_of_Referrals,Tenure_in_Months,Offer,Phone_Service,Avg_Monthly_Long_Distance_Charges,Multiple_Lines,...,Contract,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,0002-ORFBO,0,Frazier Park,93225,2,9,None,Yes,42.39,No,...,One Year,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,No Feedback Given
1,0003-MKNFE,0,Glendale,91206,0,9,None,Yes,10.69,Yes,...,Month-to-Month,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,No Feedback Given
2,0004-TLHLJ,0,Costa Mesa,92627,0,4,Offer E,Yes,33.65,No,...,Month-to-Month,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,0,Martinez,94553,1,13,Offer D,Yes,27.82,No,...,Month-to-Month,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,0,Camarillo,93010,3,3,None,Yes,7.38,No,...,Month-to-Month,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


##### 3.4 Share dataframe to tableau

In [13]:
# Connect with Google Sheets to access the data from tableau
# First, connect with Google Cloud - Google Sheets and Google Drive APIs

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('maven-churn-challenge-356910-4fe5a3e8b858.json', scopes=scope)
gc = gspread.authorize(credentials)


In [ ]:
# Send dataframe to sheets
spreadsheet_key='1RLhkE5939hLzN3LnbGSoMfwXCKs4zvZJErroSF7oXIk'
worksheet_name= "Master"
result = d2g.upload(new_mavencustomer_df, spreadsheet_key, worksheet_name, credentials=credentials, row_names=True)
print(result)